In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, DecimalType
from pyspark.sql import Row
from datetime import date
from pyspark.sql.window import Window
from pyspark.sql.functions import sum

---------------------------------------------------------------------------
ImportError                               Traceback (most recent call last)
File <command-1628525806018062>:5
      3 from datetime import date
      4 from pyspark.sql.window import Window
----> 5 from pyspark.sql.functions import sum, first_value

ImportError: cannot import name 'first_value' from 'pyspark.sql.functions' (/databricks/spark/python/pyspark/sql/functions.py)

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, DecimalType
from datetime import date
from decimal import Decimal

transactions_data = [
    (1, date(2011, 1, 1), Decimal("500.00")),
    (1, date(2011, 1, 15), Decimal("50.00")),
    (1, date(2011, 1, 22), Decimal("250.00")),
    (1, date(2011, 1, 24), Decimal("75.00")),
    (1, date(2011, 1, 26), Decimal("125.00")),
    (1, date(2011, 1, 28), Decimal("175.00")),
    (2, date(2011, 1, 1), Decimal("500.00")),
    (2, date(2011, 1, 15), Decimal("50.00")),
    (2, date(2011, 1, 22), Decimal("25.00")),
    (2, date(2011, 1, 23), Decimal("125.00")),
    (2, date(2011, 1, 26), Decimal("200.00")),
    (2, date(2011, 1, 29), Decimal("250.00")),
    (3, date(2011, 1, 1), Decimal("500.00")),
    (3, date(2011, 1, 15), Decimal("50.00")),
    (3, date(2011, 1, 22), Decimal("5000.00")),
    (3, date(2011, 1, 25), Decimal("550.00")),
    (3, date(2011, 1, 27), Decimal("95.00")),
    (3, date(2011, 1, 30), Decimal("2500.00")),
]

transactions_schema = StructType([
    StructField("AccountId", IntegerType()),
    StructField("TranDate", DateType()),
    StructField("TranAmt", DecimalType(8, 2))
])

transactions_df = spark.createDataFrame(transactions_data, schema=transactions_schema)
transactions_df.write.mode("overwrite").saveAsTable("Sample.Transactions")

window_spec = Window.partitionBy("AccountId").orderBy("TranDate").rowsBetween(Window.unboundedPreceding, Window.currentRow)

display(transactions_df)


AccountId,TranDate,TranAmt
1,2011-01-01,500.00
1,2011-01-15,50.00
1,2011-01-22,250.00
1,2011-01-24,75.00
1,2011-01-26,125.00
1,2011-01-28,175.00
2,2011-01-01,500.00
2,2011-01-15,50.00
2,2011-01-22,25.00
2,2011-01-23,125.00


In [0]:
transactions_df_1 = transactions_df.withColumn("RunTotalAmt",sum("TranAmt").over(window_spec))\
.withColumn("RunAvg",avg("TranAmt").over(window_spec))\
.withColumn("RunTraQty",count("TranAmt").over(window_spec))\
.withColumn("RunSmallAmt",min("tranAmt").over(window_spec))\
.withColumn("RunLargeAmt",max("tranAmt").over(window_spec))
display(transactions_df_1.orderBy("AccountId", "TranDate"))

AccountId,TranDate,TranAmt,RunTotalAmt,RunAvg,RunTraQty,RunSmallAmt,RunLargeAmt
1,2011-01-01,500.00,500.00,500.000000,1,500.00,500.00
1,2011-01-15,50.00,550.00,275.000000,2,50.00,500.00
1,2011-01-22,250.00,800.00,266.666667,3,50.00,500.00
1,2011-01-24,75.00,875.00,218.750000,4,50.00,500.00
1,2011-01-26,125.00,1000.00,200.000000,5,50.00,500.00
1,2011-01-28,175.00,1175.00,195.833333,6,50.00,500.00
2,2011-01-01,500.00,500.00,500.000000,1,500.00,500.00
2,2011-01-15,50.00,550.00,275.000000,2,50.00,500.00
2,2011-01-22,25.00,575.00,191.666667,3,25.00,500.00
2,2011-01-23,125.00,700.00,175.000000,4,25.00,500.00


In [0]:
sliding_window = Window.partitionBy("AccountId").orderBy("TranDate").rowsBetween(-2, 0)
rownum_window = Window.partitionBy("AccountId").orderBy("TranDate")

transactions_df_2 = transactions_df \
    .withColumn("SlideAvg", avg("TranAmt").over(sliding_window)) \
    .withColumn("SlideQty", count("*").over(sliding_window)) \
    .withColumn("SlideMin", min("TranAmt").over(sliding_window)) \
    .withColumn("SlideMax", max("TranAmt").over(sliding_window)) \
    .withColumn("SlideTotal", sum("TranAmt").over(sliding_window)) \
    .withColumn("RN", row_number().over(rownum_window))

display(transactions_df_2)

AccountId,TranDate,TranAmt,SlideAvg,SlideQty,SlideMin,SlideMax,SlideTotal,RN
1,2011-01-01,500.00,500.000000,1,500.00,500.00,500.00,1
1,2011-01-15,50.00,275.000000,2,50.00,500.00,550.00,2
1,2011-01-22,250.00,266.666667,3,50.00,500.00,800.00,3
1,2011-01-24,75.00,125.000000,3,50.00,250.00,375.00,4
1,2011-01-26,125.00,150.000000,3,75.00,250.00,450.00,5
1,2011-01-28,175.00,125.000000,3,75.00,175.00,375.00,6
2,2011-01-01,500.00,500.000000,1,500.00,500.00,500.00,1
2,2011-01-15,50.00,275.000000,2,50.00,500.00,550.00,2
2,2011-01-22,25.00,191.666667,3,25.00,500.00,575.00,3
2,2011-01-23,125.00,66.666667,3,25.00,125.00,200.00,4


In [0]:

logical_df = spark.table("Sample.Logical")

# Okno z ROWS
rows_window = Window.orderBy("Salary").rowsBetween(Window.unboundedPreceding, Window.currentRow)

# PySpark nie wspiera RANGE w pełni – efekt będzie taki sam jak ROWS jeśli Salary jest unikalne
range_window = Window.orderBy("Salary").rangeBetween(Window.unboundedPreceding, Window.currentRow)

# Dodaj kolumny
logical_df = logical_df \
    .withColumn("SumByRows", sum("Salary").over(rows_window)) \
    .withColumn("SumByRange", sum("Salary").over(range_window))

# Wyświetlenie
display(logical_df.orderBy("RowID"))

RowID,FName,Salary,SumByRows,SumByRange
1,George,800,800,800
2,Sam,950,1750,1750
3,Diane,1100,2850,2850
4,Nicholas,1250,4100,5350
5,Samuel,1250,5350,5350
6,Patricia,1300,6650,6650
7,Brian,1500,8150,8150
8,Thomas,1600,9750,9750
9,Fran,2450,12200,12200
10,Debbie,2850,15050,15050


In [0]:
#from pyspark.sql.functions import lead, lag, first_value, last_value, row_number

window_spec = Window.partitionBy("AccountId").orderBy("TranDate")

df_windowed = transactions_df.withColumn("PrevTranAmt", lag("TranAmt", 1).over(window_spec)) \
    .withColumn("NextTranAmt", lead("TranAmt", 1).over(window_spec)) \
    .withColumn("FirstTranAmt", first("TranAmt").over(window_spec)) \
    .withColumn("LastTranAmt", last("TranAmt").over(window_spec)) \
    .withColumn("RowNum", row_number().over(window_spec))
display(df_windowed)

AccountId,TranDate,TranAmt,PrevTranAmt,NextTranAmt,FirstTranAmt,LastTranAmt,RowNum
1,2011-01-01,500.00,null,50.00,500.00,500.00,1
1,2011-01-15,50.00,500.00,250.00,500.00,50.00,2
1,2011-01-22,250.00,50.00,75.00,500.00,250.00,3
1,2011-01-24,75.00,250.00,125.00,500.00,75.00,4
1,2011-01-26,125.00,75.00,175.00,500.00,125.00,5
1,2011-01-28,175.00,125.00,null,500.00,175.00,6
2,2011-01-01,500.00,null,50.00,500.00,500.00,1
2,2011-01-15,50.00,500.00,25.00,500.00,50.00,2
2,2011-01-22,25.00,50.00,125.00,500.00,25.00,3
2,2011-01-23,125.00,25.00,200.00,500.00,125.00,4
